In [18]:
from tensorflow.keras.layers import Input, LSTM, GRU, SimpleRNN, Dense, GlobalMaxPool1D,Reshape,Dropout,Lambda
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras import layers,models,Sequential
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import cross_val_score
# from lightgbm import LGBMClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import KFold
# from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [36]:
files_path = 'CleanFiles/'
files_format = '.csv'
files_postfix = 'Extract'
# file_names = ['AgiaParaskevi','Aristotelous','Elefsina',
#               'Lykovrisi','Marousi','NeaSmirni','Peristeri',
#               'Pireus','Thrakomakedones']
file_names = ['NeaSmirni']

In [37]:
def pm10_index(val):
    if val <= 25.0:
        return 0
    elif 26.0 <= val <= 50.0:
        return 1
    elif 51.0 <= val <= 90.0:
        return 2
    elif 91.0 <= val <= 180.0:
        return 3
    else:
        return 4

In [38]:
def get_time(tmp):
    return tmp[11:13]

In [39]:
def get_season(tmp):
    if tmp == 'Spring':
        return '1'
    elif tmp == 'Spring/Summer':
        return '2'
    elif tmp == 'Summer':
        return '3'
    elif tmp == 'Summer/Autumn':
        return '4'
    elif tmp == 'Autumn':
        return '5'
    elif tmp == 'Autumn/Winter':
        return '6'
    elif tmp == 'Winter/Spring':
        return '7'
    else:
        return '8'

In [40]:
def get_winddir(tmp):
    if tmp == 'N':
        return '1'
    elif tmp == 'NNE':
        return '2'
    elif tmp == 'NE':
        return '3'
    elif tmp == 'ENE':
        return '4'
    elif tmp == 'E':
        return '5'
    elif tmp == 'ESE':
        return '6'
    elif tmp == 'SE':
        return '7'
    elif tmp == 'SSE':
        return '8'
    elif tmp == 'S':
        return '9'
    elif tmp == 'SSW':
        return '10'
    elif tmp == 'SW':
        return '11'
    elif tmp == 'WSW':
        return '12'
    elif tmp == 'W':
        return '13'
    elif tmp == 'WNW':
        return '14'
    elif tmp == 'NW':
        return '15'
    else:
        return '16'

In [41]:
#neasmirni = pd.read_csv('CleanFiles/NeaSmirniExtract.csv', sep=',')
#neasmirni = neasmirni[['date_time','station_id','season','real_temp','real_windspd','real_winddir','real_humidity','forecast_tempC','forecast_windSpeed','forecast_windDirection','forecast_humidity','pm10']]
#neasmirni['pm10'] = neasmirni['pm10'].map(lambda a: pm10_index(a))
#temp = pd.read_csv('CleanFiles/PeristeriExtract.csv', sep=',')
#agiaparaskevi = pd.read_csv('CleanFiles/AgiaParaskeviExtract.csv', sep=',')

In [42]:
# T = 72
# D = 1
# X = []
# Y = []
# for t in range(len(neasmirni) - T):
#     x = neasmirni[t:t+T]
#     X.append(x)
#     y = neasmirni[t:t+T]
#     Y.append(y)

In [43]:
def create_xy(series, series2, window_size, prediction_horizon, shuffle = False):
    x = []
    y = []
    for i in range(0, len(series)):
        if len(series[(i + window_size):(i + window_size + prediction_horizon)]) < prediction_horizon:
            break
        x.append(np.array(series[i:(i + window_size)]))
        y.append(np.array(series2[(i + window_size):(i + window_size + prediction_horizon)]))
    x = np.array(x)
    y = np.array(y)
    return x,y

In [44]:
parameters = {
    'n_estimators': 2000,
    'max_depth': 4,
    'num_leaves': 2**4,
    'learning_rate': 0.1,
    'boosting_type': 'dart'
}

In [45]:
def smape(actual, predicted):
    if not all([isinstance(actual, np.ndarray), 
                isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual),
        np.array(predicted)
  
    return round(
        np.mean(
            np.abs(predicted - actual) / 
            ((np.abs(predicted) + np.abs(actual))/2)
        )*100, 2
    )

In [46]:
def gru(units,drop,features, window, horizon):
    encoder_inputs = layers.Input(shape=(window,features))
    encoder = GRU(units, dropout=drop,return_state=True)
    _,encoder_states = encoder(encoder_inputs)
    decoder=layers.RepeatVector(horizon)(encoder_states)
    out = layers.TimeDistributed(Dense(1))(decoder)
    model = models.Model(encoder_inputs, out)
    model.compile(loss='mse', optimizer=RMSprop())
    return model

In [126]:
def seq2seq(window, horizon, units,bn,drop,feautures,channel=1):
    encoder_inputs = layers.Input(shape=(window,feautures))

    encoder = GRU(units, dropout=drop,return_state=True)
    _,encoder_states = encoder(encoder_inputs)
    if bn:
        encoder_states=layers.BatchNormalization()(encoder_states)
    decoder=layers.RepeatVector(horizon)(encoder_states)
    decoder_gru = GRU(units, dropout=drop, return_sequences=True, return_state=False)
    decoder = decoder_gru(decoder, initial_state=encoder_states)
    
    out = layers.TimeDistributed(Dense(channel,use_bias=True))(decoder)
    model = models.Model(encoder_inputs, out)
    model.compile(loss='mse', optimizer=RMSprop(),metrics=['mean_squared_error'])
    return model

In [97]:
def gatedDNN(units,features, horizon):
#     model = Sequential()
#     model.add(layers.Input(shape=(72,features)))
#     model.add(layers.Dense(2, activation='relu'))
#     model.compile(loss='mean_squared_error', optimizer='adam')
#     return model

    model = Sequential([
    Lambda(lambda x: x[:, -1:, :]),
    Dense(units, activation='relu',use_bias=True),
    Dense(horizon*1),
    Reshape([horizon, 1])
    ])
    model.compile(loss='mse', optimizer=RMSprop(),metrics=['mean_squared_error'])
    return model

In [49]:
def lstm(features, window, horizon):
    model = Sequential([
    LSTM(10, input_shape=(window, features), return_sequences=True),
    Dropout(0.5),
    LSTM(10, return_sequences=True),
    Dense(horizon),
    Reshape([horizon, 1])
    ])
    
    model.compile(optimizer=RMSprop(), loss='mse', metrics=['mean_squared_error'])
    return model

In [50]:
def lightgbm():
    model = lgb.LGBMRegressor(first_metric_only = True)
    return model

In [127]:
def loadToModel(files, formating, pathFiles, postfx, params):
    T = 72
    D = 1
    X = []
    Y = []
    for fileName in files:
        window = 72 
        horizon = 48
        accuracy = 0
        #df = pd.read_csv(pathFiles + fileName + postfx + formating, sep=',')
        df = pd.read_csv(pathFiles + fileName + formating, sep=',')
        df['date_time'] = df['date_time'].astype("|S")
        df['season'] = df['season'].astype("|S")
        df['date_time'] = df['date_time'].map(lambda a: get_time(a))
        df['date_time'] = df['date_time'].astype(int)
        df['season'] = df['season'].map(lambda a: get_season(a))
        df['season'] = df['season'].astype(np.float64)
        df['forecast_windDirection'] = df['forecast_windDirection'].astype(np.float64)
        df['station_id'] = df['station_id'].astype(int)
        df2 = df[['pm10']]

        scaler = MinMaxScaler(feature_range=(-3, 3))
        names = ['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10']
        df[['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10']] = scaler.fit_transform(df[['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10']])
        
        
        scaler1 = MinMaxScaler(feature_range=(-3, 3))
        df2 = scaler1.fit_transform(df2.values)
        

        X,y = create_xy(df[['station_id','date_time','forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10']],df2,window, horizon)
        accuracy = 0.0
        
        print(fileName + ' begins prediction')
        model = seq2seq(72,48,50,True,0.5,7,1)
#         model = gru(5,0.5,7, 72, 48)
#         model = lstm(5, window, horizon)
#         model = lightgbm()
#         model = gatedDNN(50,7, horizon)

        kf = KFold(n_splits=5)

#         print(X.shape)
#         print(y.shape)
        
        epochs = 2
        batch_size = 32

        predicts = []
        tests = []
#         for trainI,testI in kf.split(X):
#             model.fit(X[trainI],y[trainI], epochs=epochs, batch_size=batch_size)
#             predictions = model.predict(X[testI])
#             predictions = scaler.inverse_transform(predictions.reshape(-1,1)).reshape(predictions.shape)
#             predicts.append(predictions)
#             tests.append(y[testI])

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
        model.fit(X_train,y_train, epochs=epochs, batch_size=batch_size)
        predictions = model.predict(X_test)
        predictions = scaler1.inverse_transform(predictions.reshape(-1,1)).reshape(predictions.shape)
        y_test = scaler1.inverse_transform(y_test.reshape(-1,1)).reshape(y_test.shape)

        return predictions, y_test

In [128]:
predicts, test = loadToModel(file_names, files_format, files_path, files_postfix, parameters)

NeaSmirni begins prediction
Train on 29298 samples
Epoch 1/2
29298/29298 [==============================] - 96s 3ms/sample - loss: 0.2390 - mean_squared_error: 0.2390
Epoch 2/2
29298/29298 [==============================] - 97s 3ms/sample - loss: 0.0324 - mean_squared_error: 0.0324


In [138]:
predicts[100]

array([[37.457363],
       [32.340134],
       [31.436138],
       [30.985289],
       [31.334944],
       [31.541927],
       [31.657084],
       [31.72189 ],
       [31.75851 ],
       [31.779161],
       [31.790651],
       [31.796818],
       [31.799871],
       [31.801006],
       [31.801094],
       [31.800512],
       [31.79961 ],
       [31.798534],
       [31.79734 ],
       [31.79612 ],
       [31.794899],
       [31.793676],
       [31.792513],
       [31.79135 ],
       [31.790274],
       [31.789227],
       [31.788237],
       [31.787306],
       [31.786375],
       [31.785532],
       [31.784718],
       [31.783903],
       [31.783146],
       [31.78242 ],
       [31.781721],
       [31.781052],
       [31.780413],
       [31.77983 ],
       [31.77925 ],
       [31.778725],
       [31.778173],
       [31.777708],
       [31.777214],
       [31.776749],
       [31.776312],
       [31.775904],
       [31.775526],
       [31.77512 ]], dtype=float32)

In [130]:
test[100]

array([[ 72.],
       [ 66.],
       [ 52.],
       [ 39.],
       [ 33.],
       [ 42.],
       [ 45.],
       [ 42.],
       [ 70.],
       [ 56.],
       [ 39.],
       [ 27.],
       [ 38.],
       [ 37.],
       [ 43.],
       [ 41.],
       [ 36.],
       [ 42.],
       [ 43.],
       [ 39.],
       [ 37.],
       [ 35.],
       [ 35.],
       [ 31.],
       [ 35.],
       [ 41.],
       [ 75.],
       [135.],
       [ 62.],
       [ 17.],
       [ 12.],
       [ 19.],
       [ 21.],
       [ 25.],
       [ 20.],
       [ 22.],
       [ 23.],
       [ 25.],
       [ 21.],
       [ 20.],
       [ 20.],
       [ 30.],
       [ 24.],
       [ 17.],
       [ 40.],
       [ 62.],
       [ 41.],
       [ 56.]])

In [71]:
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [ ]:
def return_axes(df):
    x = np.arange(df.shape[0])[:, None, None]
    y = np.arange(df.shape[1])[None, :, None]
    z = np.arange(df.shape[2])[None, None, :]
    x, y, z = np.broadcast_arrays(x, y, z)
    return x,y,z

In [ ]:
# x, y, z = return_axes(predicts)
# fig = plt.figure()
# ax = fig.gca(projection='3d')
# ax.scatter(x.ravel(),
#            y.ravel(),
#            z.ravel())

In [ ]:
# x, y, z = return_axes(test)
# fig = plt.figure()
# ax = fig.gca(projection='3d')
# ax.scatter(x.ravel(),
#            y.ravel(),
#            z.ravel())